In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/CDC_DS_PROJECT'

This code processes a folder of images in batches to extract both CNN-based features and simple hand-crafted image statistics, while using a checkpoint system to avoid reprocessing. It loads a ResNet50 model (without the top layer) and, for each batch of images, computes the CNN embeddings along with statistical summaries like mean, standard deviation, min, and max of the features. It also calculates green_ratio, water_ratio, and edge_density from the raw image. All results are stored in lists and periodically saved to a checkpoint file (.npz) so processing can resume if interrupted. The code efficiently skips already processed images, handles errors, and ensures that every image contributes both high-level deep features and interpretable low-level features for downstream analysis.

In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

#path assigning

IMAGE_DIR = f"{path}/TRAIN_IMAGES"
TARGET_SIZE = (224, 224)
BATCH_SIZE = 32
CHECKPOINT_DIR = f"{path}/image_feature_checkpoint"
CHECKPOINT_FILE = os.path.join(CHECKPOINT_DIR, "train_checkpoint")

# Load CNN model

cnn_model = ResNet50(weights="imagenet", include_top=False, pooling="avg")

# Custom feature functions

def green_ratio(img):
    return img[..., 1].mean() / 255.0

def water_ratio(img):
    return img[..., 2].mean() / 255.0

def edge_density(img):
    return img.mean() / 255.0

# Load checkpoint if exists
if os.path.exists(CHECKPOINT_FILE):
    print("Loading checkpoint...")
    data = np.load(CHECKPOINT_FILE, allow_pickle=True)
    image_embeddings = list(data["image_embeddings"])
    image_ids        = list(data["image_ids"])
    cnn_mean_vals    = list(data["cnn_mean_vals"])
    cnn_std_vals     = list(data["cnn_std_vals"])
    cnn_max_vals     = list(data["cnn_max_vals"])
    cnn_min_vals     = list(data["cnn_min_vals"])
    green_vals       = list(data["green_vals"])
    water_vals       = list(data["water_vals"])
    edge_vals        = list(data["edge_vals"])
    processed_ids = set(image_ids)

    print(f" Resumed from checkpoint: {len(processed_ids)} images already processed")

else:
    print(" No checkpoint found. Starting fresh.")

    image_embeddings = []
    image_ids = []
    cnn_mean_vals = []
    cnn_std_vals = []
    cnn_max_vals = []
    cnn_min_vals = []
    green_vals = []
    water_vals = []
    edge_vals = []

    processed_ids = set()


# Get image files

image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.endswith(".jpg")])
total_images = len(image_files)

# Batch generator

def batch_generator(files, batch_size):
    for i in range(0, len(files), batch_size):
        yield i // batch_size + 1, files[i:i + batch_size]

total_batches = (total_images + BATCH_SIZE - 1) // BATCH_SIZE

# Main processing loop

for batch_num, batch_files in tqdm(
        batch_generator(image_files, BATCH_SIZE),
        total=total_batches,
        desc="Processing batches"):
    batch_imgs = []
    batch_raw = []
    batch_ids = []

    print(f"\n🔹 Loading batch {batch_num}/{total_batches}")

    for img_name in batch_files:
        try:
            property_id = int(img_name.split("_")[1].split(".")[0])

            # Skip already processed images
            if property_id in processed_ids:
                continue

            img_path = os.path.join(IMAGE_DIR, img_name)

            # Load image ONCE
            img = load_img(img_path, target_size=TARGET_SIZE)
            img_arr = img_to_array(img)

            batch_imgs.append(img_arr)
            batch_raw.append(img_arr.astype(np.uint8))
            batch_ids.append(property_id)

        except Exception as e:
            print("Skipping:", img_name, "Reason:", e)

    if len(batch_imgs) == 0:
        print("⏭ No new images in this batch")
        continue

    # Prepare CNN batch
    batch_imgs = preprocess_input(np.array(batch_imgs))

    # CNN forward pass
    features_batch = cnn_model(batch_imgs, training=False).numpy()

    # Store results
    for features, raw_img, pid in zip(features_batch, batch_raw, batch_ids):
        image_embeddings.append(features)
        cnn_mean_vals.append(features.mean())
        cnn_std_vals.append(features.std())
        cnn_max_vals.append(features.max())
        cnn_min_vals.append(features.min())
        image_ids.append(pid)
        green_vals.append(green_ratio(raw_img))
        water_vals.append(water_ratio(raw_img))
        edge_vals.append(edge_density(raw_img))
        processed_ids.add(pid)

    # SAVE CHECKPOINT AFTER EACH BATCH

    np.savez(
        CHECKPOINT_FILE,
        image_embeddings=np.array(image_embeddings),
        image_ids=np.array(image_ids),
        cnn_mean_vals=np.array(cnn_mean_vals),
        cnn_std_vals=np.array(cnn_std_vals),
        cnn_max_vals=np.array(cnn_max_vals),
        cnn_min_vals=np.array(cnn_min_vals),
        green_vals=np.array(green_vals),
        water_vals=np.array(water_vals),
        edge_vals=np.array(edge_vals),
    )

    print(f" Checkpoint saved and images processed")

print("Processing complete")



94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
 No checkpoint found. Starting fresh.


Processing batches:   0%|          | 0/478 [00:00<?, ?it/s]


🔹 Loading batch 1/478


Processing batches:   0%|          | 1/478 [00:19<2:35:33, 19.57s/it]

 Checkpoint saved and images processed

🔹 Loading batch 2/478


...

 Checkpoint saved and images processed

🔹 Loading batch 477/478


Processing batches: 100%|█████████▉| 477/478 [1:11:17<00:11, 11.62s/it]

 Checkpoint saved and images processed

🔹 Loading batch 478/478


Processing batches: 100%|██████████| 478/478 [1:11:21<00:00,  8.96s/it]

 Checkpoint saved and images processed
Processing complete


Creating a dataframe of saved images features

In [ ]:
import pandas as pd
df_features_image = pd.DataFrame({
    "image_id": image_ids,
    "cnn_mean": cnn_mean_vals,
    "cnn_std": cnn_std_vals,
    "cnn_max": cnn_max_vals,
    "cnn_min": cnn_min_vals,
    "green_ratio": green_vals,
    "water_ratio": water_vals,
    "edge_density": edge_vals
})

In [ ]:
df_features_image

,image_id,cnn_mean,cnn_std,cnn_max,cnn_min,green_ratio,water_ratio,edge_density
0,1000102,0.563088,0.810735,8.721620,0.0,0.526612,0.494444,0.510726
1,1001200050,0.447758,0.773688,9.206857,0.0,0.392955,0.341563,0.367749
2,1003000175,0.513795,0.725000,7.410271,0.0,0.389056,0.350668,0.366403
3,100300280,0.507359,0.888578,9.628031,0.0,0.398349,0.377876,0.382276
4,100300530,0.521638,0.768914,8.762383,0.0,0.389360,0.388364,0.379537
...,...,...,...,...,...,...,...,...
15260,993002127,0.445838,0.846385,9.758054,0.0,0.368202,0.369664,0.358583
15261,993002177,0.516960,0.788895,8.020640,0.0,0.319089,0.324439,0.313251
15262,993002225,0.410236,0.693904,7.623897,0.0,0.389442,0.356556,0.367497
15263,993002247,0.520062,0.854535,8.621179,0.0,0.307145,0.313791,0.301887


In [ ]:
df_features_image.to_csv(f'{path}/image_features.csv')#saving csv file of image features

In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

#path assigning
IMAGE_DIR = f"{path}/TEST_IMAGES2"
TARGET_SIZE = (224, 224)
BATCH_SIZE = 32
CHECKPOINT_DIR = f"{path}/image_feature_checkpoint"
CHECKPOINT_FILE = os.path.join(CHECKPOINT_DIR, "test_checkpoint")

#load cnn model
cnn_model = ResNet50(weights="imagenet", include_top=False, pooling="avg")

#defining features
def green_ratio(img):
    return img[..., 1].mean() / 255.0

def water_ratio(img):
    return img[..., 2].mean() / 255.0

def edge_density(img):
    return img.mean() / 255.0

#check if checkpoints exists
if os.path.exists(CHECKPOINT_FILE):
    print("Loading checkpoint...")
    data = np.load(CHECKPOINT_FILE, allow_pickle=True)
    image_embeddings = list(data["image_embeddings"])
    image_ids        = list(data["image_ids"])
    cnn_mean_vals    = list(data["cnn_mean_vals"])
    cnn_std_vals     = list(data["cnn_std_vals"])
    cnn_max_vals     = list(data["cnn_max_vals"])
    cnn_min_vals     = list(data["cnn_min_vals"])
    green_vals       = list(data["green_vals"])
    water_vals       = list(data["water_vals"])
    edge_vals        = list(data["edge_vals"])
    processed_ids = set(image_ids)

    print(f" Resumed from checkpoint and images processed")

else:
    print("No checkpoint found. Starting fresh.")
    image_embeddings = []
    image_ids = []
    cnn_mean_vals = []
    cnn_std_vals = []
    cnn_max_vals = []
    cnn_min_vals = []
    green_vals = []
    water_vals = []
    edge_vals = []

    processed_ids = set()

image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.endswith(".jpg")])
total_images = len(image_files)

def batch_generator(files, batch_size):
    for i in range(0, len(files), batch_size):
        yield i // batch_size + 1, files[i:i + batch_size]

total_batches = (total_images + BATCH_SIZE - 1) // BATCH_SIZE

for batch_num, batch_files in tqdm(
        batch_generator(image_files, BATCH_SIZE),
        total=total_batches,
        desc="Processing batches"):

    batch_imgs = []
    batch_raw = []
    batch_ids = []

    print(f"\n Loading batch {batch_num}/{total_batches}")

    for img_name in batch_files:
        try:
            property_id = int(img_name.split("_")[1].split(".")[0])

            # Skip already processed images
            if property_id in processed_ids:
                continue

            img_path = os.path.join(IMAGE_DIR, img_name)

            # Load image ONCE
            img = load_img(img_path, target_size=TARGET_SIZE)
            img_arr = img_to_array(img)

            batch_imgs.append(img_arr)
            batch_raw.append(img_arr.astype(np.uint8))
            batch_ids.append(property_id)

        except Exception as e:
            print("Skipping:", img_name, "Reason:", e)

    if len(batch_imgs) == 0:
        print("No new images in this batch")
        continue

    # Prepare CNN batch
    batch_imgs = preprocess_input(np.array(batch_imgs))

    # CNN forward pass
    features_batch = cnn_model(batch_imgs, training=False).numpy()

    # Store results
    for features, raw_img, pid in zip(features_batch, batch_raw, batch_ids):
        image_embeddings.append(features)
        cnn_mean_vals.append(features.mean())
        cnn_std_vals.append(features.std())
        cnn_max_vals.append(features.max())
        cnn_min_vals.append(features.min())
        image_ids.append(pid)

        green_vals.append(green_ratio(raw_img))
        water_vals.append(water_ratio(raw_img))
        edge_vals.append(edge_density(raw_img))

        processed_ids.add(pid)


    # SAVE CHECKPOINT AFTER EACH BATCH
    np.savez(
        CHECKPOINT_FILE,
        image_embeddings=np.array(image_embeddings),
        image_ids=np.array(image_ids),
        cnn_mean_vals=np.array(cnn_mean_vals),
        cnn_std_vals=np.array(cnn_std_vals),
        cnn_max_vals=np.array(cnn_max_vals),
        cnn_min_vals=np.array(cnn_min_vals),
        green_vals=np.array(green_vals),
        water_vals=np.array(water_vals),
        edge_vals=np.array(edge_vals),
    )

    print("Checkpoint saved")

print("\nprocessing complete")

No checkpoint found. Starting fresh.


Processing batches:   0%|          | 0/169 [00:00<?, ?it/s]


 Loading batch 1/169


Processing batches:   1%|          | 1/169 [00:18<52:14, 18.66s/it]

Checkpoint saved

 Loading batch 2/169


...

Checkpoint saved

 Loading batch 168/169


Processing batches:  99%|█████████▉| 168/169 [22:13<00:07,  7.19s/it]

Checkpoint saved

 Loading batch 169/169


Processing batches: 100%|██████████| 169/169 [22:17<00:00,  7.91s/it]

Checkpoint saved

processing complete


saving features in a dataframe

In [ ]:
import pandas as pd
df_features_image_test = pd.DataFrame({
    "image_id": image_ids,
    "cnn_mean": cnn_mean_vals,
    "cnn_std": cnn_std_vals,
    "cnn_max": cnn_max_vals,
    "cnn_min": cnn_min_vals,
    "green_ratio": green_vals,
    "water_ratio": water_vals,
    "edge_density": edge_vals
})

In [ ]:
df_features_image_test.to_csv(f'{path}/test_features.csv')#saving a csv file

# Features explaination

cnn_mean:
The average value of all CNN-extracted features. It represents the overall strength or presence of learned visual patterns in the image.

cnn_std:
Measures how much the CNN features vary across the image. Higher values indicate more visual diversity and complex patterns.

cnn_max:
The maximum value among CNN features. It captures the strongest activated visual pattern detected by the CNN.

cnn_min:
The minimum value among CNN features. It represents the weakest or least activated visual response in the image.

green_ratio:
The proportion of pixels dominated by green color. It is commonly used to estimate vegetation or plant cover in the image.

water_ratio:
The proportion of pixels identified as water-like based on color characteristics. It helps detect rivers, lakes, or other water bodies.

edge_density:
The ratio of edge pixels to total pixels in the image. Higher edge density indicates more texture, boundaries, or structural complexity.

This was done to make learning easier and more reliable by keeping only what matters most.
By compressing and summarizing, repetition and noise are reduced while important patterns remain.
With less to process, training becomes faster, results generalize better, and instability is avoided.
